In [300]:
import pandas as pd
import xgboost as xgb
import numpy as np
import datetime as dt
import random
from sportsreference.ncaab.teams import Teams
from sklearn.preprocessing import scale
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 100)

In [4]:
teams_df = pd.DataFrame()
i = 2010
while i <=2019:
    print("Downloading data for the", i, "season.")
    start = dt.datetime.now()
    teams = Teams(year = i)
    end = dt.datetime.now()
    teams_temp = teams.dataframes
    teams_temp['Season'] = i
    teams_df = pd.concat([teams_df, teams_temp])
    span = end - start
    print(i, "took", span, "seconds to download.")
    i += 1

2010 took 0:00:20.515166 seconds to download.
2011 took 0:00:20.536113 seconds to download.
2012 took 0:00:20.277801 seconds to download.
2013 took 0:00:20.556057 seconds to download.
2014 took 0:00:20.619887 seconds to download.
2015 took 0:00:20.638837 seconds to download.
2016 took 0:00:20.265835 seconds to download.
2017 took 0:00:20.450341 seconds to download.
2018 took 0:00:21.563367 seconds to download.
2019 took 0:00:22.422071 seconds to download.


In [6]:
teams_df.head()

,abbreviation,assist_percentage,assists,away_losses,away_wins,block_percentage,blocks,conference,conference_losses,conference_wins,...,two_point_field_goal_percentage,two_point_field_goals,total_rebound_percentage,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,win_percentage,wins,Season
AIR-FORCE,AIR-FORCE,61.6,389,10,0,6.2,51,mwc,15,1,...,0.527,456,46.8,851,0.529,19.0,394,0.323,10,2010
AKRON,AKRON,53.9,473,4,7,8.5,114,mac,4,12,...,0.483,642,51.6,1269,0.521,16.4,464,0.686,24,2010
ALABAMA-AM,ALABAMA-AM,48.1,288,9,3,12.7,143,swac,10,8,...,0.410,491,46.8,1004,0.463,18.8,444,0.407,11,2010
ALABAMA-BIRMINGHAM,ALABAMA-BIRMINGHAM,51.1,390,4,9,7.3,91,cusa,5,11,...,0.474,586,53.6,1248,0.518,17.0,451,0.735,25,2010
ALABAMA-STATE,ALABAMA-STATE,60.0,400,11,6,11.1,129,swac,6,12,...,0.448,477,51.3,1101,0.499,20.0,501,0.516,16,2010


In [124]:
teams_df.columns

Index(['abbreviation', 'assist_percentage', 'assists', 'away_losses',
       'away_wins', 'block_percentage', 'blocks', 'conference',
       'conference_losses', 'conference_wins', 'defensive_rebounds',
       'effective_field_goal_percentage', 'field_goal_attempts',
       'field_goal_percentage', 'field_goals', 'free_throw_attempt_rate',
       'free_throw_attempts', 'free_throw_percentage', 'free_throws',
       'free_throws_per_field_goal_attempt', 'games_played', 'home_losses',
       'home_wins', 'losses', 'minutes_played', 'name', 'net_rating',
       'offensive_rating', 'offensive_rebound_percentage',
       'offensive_rebounds', 'opp_assist_percentage', 'opp_assists',
       'opp_block_percentage', 'opp_blocks', 'opp_defensive_rebounds',
       'opp_effective_field_goal_percentage', 'opp_field_goal_attempts',
       'opp_field_goal_percentage', 'opp_field_goals',
       'opp_free_throw_attempt_rate', 'opp_free_throw_attempts',
       'opp_free_throw_percentage', 'opp_free_thro

In [135]:
from sportsreference.ncaab.schedule import Schedule

purdue_schedule = Schedule('ARIZONA')
for game in purdue_schedule:
    boxscore = game.boxscore
    print(game.boxscore_index)
#df = purdue_schedule.dataframe

2019-11-06-21-arizona
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [60]:
game_data = Boxscore('2019-11-06-19-purdue')
df = game_data.dataframe

In [63]:
cols = ['away_assist_percentage', 'away_assists', 'away_block_percentage',
       'away_blocks', 'away_defensive_rating',
       'away_defensive_rebound_percentage', 'away_defensive_rebounds',
       'away_effective_field_goal_percentage', 'away_field_goal_attempts',
       'away_field_goal_percentage', 'away_field_goals',
       'away_free_throw_attempt_rate', 'away_free_throw_attempts',
       'away_free_throw_percentage', 'away_free_throws', 'away_losses',
       'away_minutes_played', 'away_offensive_rating',
       'away_offensive_rebound_percentage', 'away_offensive_rebounds',
       'away_personal_fouls', 'away_points', 'away_ranking',
       'away_steal_percentage', 'away_steals', 'away_three_point_attempt_rate',
       'away_three_point_field_goal_attempts',
       'away_three_point_field_goal_percentage',
       'away_three_point_field_goals', 'away_total_rebound_percentage',
       'away_total_rebounds', 'away_true_shooting_percentage',
       'away_turnover_percentage', 'away_turnovers',
       'away_two_point_field_goal_attempts',
       'away_two_point_field_goal_percentage', 'away_two_point_field_goals',
       'away_win_percentage', 'away_wins', 'date', 'home_assist_percentage',
       'home_assists', 'home_block_percentage', 'home_blocks',
       'home_defensive_rating', 'home_defensive_rebound_percentage',
       'home_defensive_rebounds', 'home_effective_field_goal_percentage',
       'home_field_goal_attempts', 'home_field_goal_percentage',
       'home_field_goals', 'home_free_throw_attempt_rate',
       'home_free_throw_attempts', 'home_free_throw_percentage',
       'home_free_throws', 'home_losses', 'home_minutes_played',
       'home_offensive_rating', 'home_offensive_rebound_percentage',
       'home_offensive_rebounds', 'home_personal_fouls', 'home_points',
       'home_ranking', 'home_steal_percentage', 'home_steals',
       'home_three_point_attempt_rate', 'home_three_point_field_goal_attempts',
       'home_three_point_field_goal_percentage',
       'home_three_point_field_goals', 'home_total_rebound_percentage',
       'home_total_rebounds', 'home_true_shooting_percentage',
       'home_turnover_percentage', 'home_turnovers',
       'home_two_point_field_goal_attempts',
       'home_two_point_field_goal_percentage', 'home_two_point_field_goals',
       'home_win_percentage', 'home_wins', 'location', 'losing_abbr',
       'losing_name', 'pace', 'winner', 'winning_abbr', 'winning_name']

In [77]:
df = pd.DataFrame(columns=cols)
for team in Teams():
    abr = team.abbreviation
    schedule = Schedule(abr)
    for game in schedule:
        print(game.boxscore_index)
        df2 = Boxscore(game.boxscore_index).dataframe
        df = df.append(df2)
        df['game_id'] = game

2019-11-05-20-abilene-christian
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
2019-11-07-21-air-force
2019-11-09-16-air-force
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
2019-11-05-19-akron
2019-11-08-19-west-virginia
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
2019-11-05-20-tennessee-state
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
2019-11-07-20-troy
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
2019-11-05-20-gonzaga
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
N

KeyboardInterrupt: 

In [87]:
teams = Teams()

In [4]:
teams = Teams(year = 2019)
for team in teams:
    print(team.abbreviation)

ABILENE-CHRISTIAN
AIR-FORCE
AKRON
ALABAMA-AM
ALABAMA-BIRMINGHAM
ALABAMA-STATE
ALABAMA
ALBANY-NY
ALCORN-STATE
AMERICAN
APPALACHIAN-STATE
ARIZONA-STATE
ARIZONA
ARKANSAS-LITTLE-ROCK
ARKANSAS-PINE-BLUFF
ARKANSAS-STATE
ARKANSAS
ARMY
AUBURN
AUSTIN-PEAY
BALL-STATE
BAYLOR
BELMONT
BETHUNE-COOKMAN
BINGHAMTON
BOISE-STATE
BOSTON-COLLEGE
BOSTON-UNIVERSITY
BOWLING-GREEN-STATE
BRADLEY
BRIGHAM-YOUNG
BROWN
BRYANT
BUCKNELL
BUFFALO
BUTLER
CAL-POLY
CAL-STATE-BAKERSFIELD
CAL-STATE-FULLERTON
CAL-STATE-NORTHRIDGE
CALIFORNIA-BAPTIST
CALIFORNIA-DAVIS
CALIFORNIA-IRVINE
CALIFORNIA-RIVERSIDE
CALIFORNIA-SANTA-BARBARA
CALIFORNIA
CAMPBELL
CANISIUS
CENTRAL-ARKANSAS
CENTRAL-CONNECTICUT-STATE
CENTRAL-FLORIDA
CENTRAL-MICHIGAN
CHARLESTON-SOUTHERN
CHARLOTTE
CHATTANOOGA
CHICAGO-STATE
CINCINNATI
CITADEL
CLEMSON
CLEVELAND-STATE
COASTAL-CAROLINA
COLGATE
COLLEGE-OF-CHARLESTON
COLORADO-STATE
COLORADO
COLUMBIA
CONNECTICUT
COPPIN-STATE
CORNELL
CREIGHTON
DARTMOUTH
DAVIDSON
DAYTON
DELAWARE-STATE
DELAWARE
DENVER
DEPAUL
DETROIT-MERCY

In [139]:
for team in teams:
    if team.abbreviation == 'ARIZONA':
        print(team.abbreviation)

ARIZONA


In [88]:
pur = pd.DataFrame(columns=cols)

schedule = Schedule('PURDUE',year = 2018)
for game in purdue_schedule:
    print(game.boxscore_index)
    df2 = Boxscore(game.boxscore_index).dataframe
    pur = pur.append(df2)
    pur['game_id'] = game
    


2019-11-06-19-purdue
2019-11-09-19-purdue
None
None
None
None
None
None
None


KeyboardInterrupt: 

In [79]:
pur.head()

,away_assist_percentage,away_assists,away_block_percentage,away_blocks,away_defensive_rating,away_defensive_rebound_percentage,away_defensive_rebounds,away_effective_field_goal_percentage,away_field_goal_attempts,away_field_goal_percentage,...,home_two_point_field_goals,home_win_percentage,home_wins,location,losing_abbr,losing_name,pace,winner,winning_abbr,winning_name
2019-11-06-19-purdue,52.2,12,2.4,1,109.7,64.3,18,0.421,63,0.365,...,25,1.0,1,"Mackey Arena, West Lafayette, Indiana",GREEN-BAY,Green Bay,72.2,Home,PURDUE,Purdue
2019-11-09-19-purdue,75.0,18,13.3,6,101.5,61.3,19,0.633,45,0.533,...,17,0.5,1,"Mackey Arena, West Lafayette, Indiana",PURDUE,Purdue,65.2,Away,TEXAS,Texas


In [218]:
teams = Teams(year = 2019)

In [148]:
dataset = pd.DataFrame()

In [ ]:
'''
    elif team.abbreviation == 'FAIRFIELD':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
        
    elif team.abbreviation == 'BALL-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
        
    elif team.abbreviation == 'APPALACHIAN-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'DAVIDSON':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'VIRGINIA-TECH':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'ROBERT-MORRIS':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
        
    elif team.abbreviation == 'FLORIDA-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'MICHIGAN':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'MARYLAND':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'TEXAS':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
        
    elif team.abbreviation == 'NOTRE-DAME':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
        
    elif team.abbreviation == 'OHIO':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'BELMONT':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'IOWA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'MICHIGAN-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'WISCONSIN':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'RUTGERS':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'INDIANA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'OHIO-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'PENN-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'MINNESOTA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'NEBRASKA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'ILLINOIS':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'NORTHWESTERN':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'OLD-DOMINION':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'VILLANOVA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'TENNESSEE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'VIRGINIA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df]) '''




or  or (team.abbreviation == 'DAVIDSON') or (team.abbreviation == 'VIRGINIA-TECH') or
    (team.abbreviation == 'ROBERT-MORRIS') or (team.abbreviation == 'TEXAS') or (team.abbreviation == 'MICHIGAN') or
    (team.abbreviation == 'MARYLAND') or (team.abbreviation == 'NOTRE-DAME') or (team.abbreviation == 'FLORIDA-STATE') or
    (team.abbreviation == 'OHIO') or (team.abbreviation == 'BELMONT') or (team.abbreviation == 'IOWA') or
    (team.abbreviation == 'MICHIGAN-STATE') or (team.abbreviation == 'WISCONSIN') or (team.abbreviation == 'RUTGERS') or
    (team.abbreviation == 'INDIANA') or (team.abbreviation == 'OHIO-STATE') or (team.abbreviation == 'PENN-STATE') or
    (team.abbreviation == 'MINNESOTA') or (team.abbreviation == 'NEBRASKA') or (team.abbreviation == 'ILLINOIS') or
    (team.abbreviation == 'NORTHWESTERN') or (team.abbreviation == 'OLD-DOMINION') or (team.abbreviation == 'VILLANOVA') or
    (team.abbreviation == 'TENNESSEE') or (team.abbreviation == 'VIRGINIA')

In [219]:
dataset = pd.DataFrame()
lst = []
#teams = Teams(year = 2019)
for team in teams:
    if team.abbreviation == 'PURDUE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
    elif team.abbreviation == 'FAIRFIELD':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
        
    elif team.abbreviation == 'BALL-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
        
    elif team.abbreviation == 'APPALACHIAN-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
    
    elif team.abbreviation == 'DAVIDSON':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
    
    elif team.abbreviation == 'VIRGINIA-TECH':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
    
    elif team.abbreviation == 'ROBERT-MORRIS':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
        
    elif team.abbreviation == 'FLORIDA-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
    
    elif team.abbreviation == 'MICHIGAN':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
    
    elif team.abbreviation == 'MARYLAND':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
    
    elif team.abbreviation == 'TEXAS':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
        
    elif team.abbreviation == 'NOTRE-DAME':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
        
    elif team.abbreviation == 'OHIO':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
    
    elif team.abbreviation == 'BELMONT':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
    
    elif team.abbreviation == 'IOWA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
    
    elif team.abbreviation == 'MICHIGAN-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
    
    elif team.abbreviation == 'WISCONSIN':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
    
    elif team.abbreviation == 'RUTGERS':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
    
    elif team.abbreviation == 'INDIANA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
    
    elif team.abbreviation == 'OHIO-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
    
    elif team.abbreviation == 'PENN-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
    
    elif team.abbreviation == 'MINNESOTA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
    
    elif team.abbreviation == 'NEBRASKA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
    
    elif team.abbreviation == 'ILLINOIS':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
    
    elif team.abbreviation == 'NORTHWESTERN':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
    
    elif team.abbreviation == 'OLD-DOMINION':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
    
    elif team.abbreviation == 'VILLANOVA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
    
    elif team.abbreviation == 'TENNESSEE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)
    
    elif team.abbreviation == 'VIRGINIA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        df.sort_values(by='date',inplace=True)
        df['game_index'] = np.arange(len(df))
        lst.append(df)

for i in lst:
    dataset = pd.concat([dataset,i])

APPALACHIAN-STATE
BALL-STATE
BELMONT
DAVIDSON
FAIRFIELD
FLORIDA-STATE
ILLINOIS
INDIANA
IOWA
MARYLAND
MICHIGAN-STATE
MICHIGAN
MINNESOTA
NEBRASKA
NORTHWESTERN
NOTRE-DAME
OHIO-STATE
OHIO
OLD-DOMINION
PENN-STATE
PURDUE
ROBERT-MORRIS
RUTGERS
TENNESSEE
TEXAS
VILLANOVA
VIRGINIA-TECH
VIRGINIA
WISCONSIN


C:\Users\socst\Anaconda3\lib\site-packages\ipykernel_launcher.py:266: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [220]:
dataset

,away_assist_percentage,away_assists,away_block_percentage,away_blocks,away_defensive_rating,away_defensive_rebound_percentage,away_defensive_rebounds,away_effective_field_goal_percentage,away_field_goal_attempts,away_field_goal_percentage,...,home_win_percentage,home_wins,location,losing_abbr,losing_name,pace,team,winner,winning_abbr,winning_name
2018-12-15-13-south-florida,40.9,9.0,12.8,6.0,102.7,59.0,23.0,0.438,56.0,0.393,...,0.778,7.0,"Sun Dome, Tampa, Florida",APPALACHIAN-STATE,Appalachian State,74.4,APPALACHIAN-STATE,Home,SOUTH-FLORIDA,South Florida
2018-12-18-18-georgetown,59.3,16.0,5.0,2.0,103.8,77.5,31.0,0.420,75.0,0.360,...,0.727,8.0,"Capital One Arena, Washington, District of Col...",APPALACHIAN-STATE,Appalachian State,80.3,APPALACHIAN-STATE,Home,GEORGETOWN,Georgetown
2018-12-20-19-appalachian-state,36.0,9.0,2.4,1.0,128.6,70.4,19.0,0.452,63.0,0.397,...,0.417,5.0,"Holmes Convocation Center, Boone, North Carolina",Milligan\n\t\t\t,Milligan\n\t\t\t,77.5,APPALACHIAN-STATE,Home,APPALACHIAN-STATE,Appalachian State
2018-12-30-15-saint-louis,42.1,8.0,2.2,1.0,122.1,57.9,22.0,0.356,59.0,0.322,...,0.692,9.0,"Chaifetz Arena, St. Louis, Missouri",APPALACHIAN-STATE,Appalachian State,68.3,APPALACHIAN-STATE,Home,SAINT-LOUIS,Saint Louis
2018-12-04-19-appalachian-state,21.9,7.0,5.0,2.0,120.5,60.7,17.0,0.472,72.0,0.444,...,0.375,3.0,"Holmes Convocation Center, Boone, North Carolina",HOWARD,Howard,82.6,APPALACHIAN-STATE,Home,APPALACHIAN-STATE,Appalachian State
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-11-22-13-oklahoma,57.1,16.0,7.9,3.0,90.6,72.4,21.0,0.593,59.0,0.475,...,0.800,4.0,"Imperial Arena at Atlantis Resort, Paradise Is...",OKLAHOMA,Oklahoma,63.7,WISCONSIN,Away,WISCONSIN,Wisconsin
2018-11-23-14-wisconsin,50.0,10.0,5.4,2.0,79.3,84.6,22.0,0.407,54.0,0.370,...,0.833,5.0,"Imperial Arena at Atlantis Resort, Paradise Is...",WISCONSIN,Wisconsin,57.6,WISCONSIN,Away,VIRGINIA,Virginia
2018-11-27-21-wisconsin,29.0,9.0,5.6,2.0,111.3,78.1,25.0,0.523,66.0,0.470,...,0.857,6.0,"Kohl Center, Madison, Wisconsin",NORTH-CAROLINA-STATE,NC State,70.7,WISCONSIN,Home,WISCONSIN,Wisconsin
2018-11-30-20-iowa,48.1,13.0,2.9,1.0,97.1,71.1,27.0,0.554,56.0,0.482,...,0.857,6.0,"Carver-Hawkeye Arena, Iowa City, Iowa",IOWA,Iowa,67.8,WISCONSIN,Away,WISCONSIN,Wisconsin


In [412]:
df = dataset.copy()

In [413]:
df.team.value_counts()

MICHIGAN-STATE       39
VIRGINIA             38
TENNESSEE            37
FLORIDA-STATE        37
TEXAS                37
MICHIGAN             37
MINNESOTA            36
NEBRASKA             36
VILLANOVA            36
PURDUE               36
IOWA                 35
INDIANA              35
OHIO-STATE           35
ROBERT-MORRIS        35
VIRGINIA-TECH        35
OLD-DOMINION         35
DAVIDSON             34
WISCONSIN            34
MARYLAND             34
ILLINOIS             33
BALL-STATE           33
BELMONT              33
NOTRE-DAME           33
APPALACHIAN-STATE    32
PENN-STATE           32
NORTHWESTERN         32
RUTGERS              31
OHIO                 31
FAIRFIELD            31
Name: team, dtype: int64

In [414]:
def get_home(row):
    
    if row.winner == 'Away':
        home = row.losing_abbr
        
    if row.winner == 'Home':
        home = row.winning_abbr
    return home

def get_away(row):
    
    if row.winner == 'Home':
        away = row.losing_abbr
        
    if row.winner == 'Away':
        away = row.winning_abbr
    return away

In [415]:
df['HomeTeam'] = df.apply (lambda row: get_home(row), axis=1)
df['AwayTeam'] = df.apply (lambda row: get_away(row), axis=1)

In [416]:
df.drop(columns=['losing_abbr','losing_name','winning_abbr','winning_name','location','home_ranking',
                 'away_ranking','home_wins','away_wins','home_losses','away_losses','home_minutes_played','away_minutes_played',
                
                'away_assists', 'away_blocks','away_defensive_rebounds','away_field_goal_attempts','away_field_goals',
                 'away_free_throw_attempts','away_free_throws','away_offensive_rebounds','away_personal_fouls', 'away_steals',
                 'away_three_point_field_goal_attempts','away_three_point_field_goals','away_total_rebounds','away_turnovers',
                 'away_two_point_field_goal_attempts','away_two_point_field_goals','home_assists', 'home_blocks',
                 'home_defensive_rebounds','home_field_goal_attempts','home_field_goals','home_free_throw_attempts','home_free_throws',
                 'home_offensive_rebounds','home_personal_fouls', 'home_steals','home_three_point_field_goal_attempts','home_three_point_field_goals',
                 'home_total_rebounds','home_turnovers','home_two_point_field_goal_attempts','home_two_point_field_goals'
                
                
                
                ],inplace=True)

In [417]:
df.sort_values(by=['team','game_index'],inplace=True)

In [418]:
df["away_free_throw_percentage"] = pd.to_numeric(df["away_free_throw_percentage"])

In [419]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1002 entries, 2018-12-15-13-south-florida to 2018-11-06-20-wisconsin
Data columns (total 45 columns):
away_assist_percentage                    1002 non-null float64
away_block_percentage                     1002 non-null float64
away_defensive_rating                     1002 non-null float64
away_defensive_rebound_percentage         1002 non-null float64
away_effective_field_goal_percentage      1002 non-null float64
away_field_goal_percentage                1002 non-null float64
away_free_throw_attempt_rate              1002 non-null float64
away_free_throw_percentage                1001 non-null float64
away_offensive_rating                     1002 non-null float64
away_offensive_rebound_percentage         1002 non-null float64
away_points                               1002 non-null float64
away_steal_percentage                     1002 non-null float64
away_three_point_attempt_rate             1002 non-null float64
away_three_point_fiel

In [420]:
def create_stats_features_ema(stats, span):
    
    # Create a copy of the DataFrame
    ema_features = stats[['team', 'winner','HomeTeam', 'AwayTeam','date','game_index','home_points','away_points']].copy()
    
    # Get the columns that we want to create EMA for
    feature_names = stats.drop(columns=['team', 'winner','HomeTeam', 'AwayTeam','date','game_index','home_points','away_points']).columns
    
    # Loop over the features
    for feature_name in feature_names:
        feature_ema = (stats.groupby('team')[feature_name] # Calculate the EMA
                                                  .transform(lambda row: row.ewm(span=span, min_periods=2)
                                                             .mean()
                                                             .shift(1))) # Shift the data down 1 so we don't leak data
        ema_features[feature_name] = feature_ema # Add the new feature to the DataFrame
    return ema_features

In [421]:
def restructure_stats_features(stats_features):
    non_features = ['team', 'winner','HomeTeam', 'AwayTeam','date','game_index']

    stats_features_restructured = (stats_features.query('HomeTeam==team')
                                    .rename(columns={col: 'f_' + col + 'Home' for col in stats_features.columns if col not in non_features})
                                    .drop(columns=['team'])
                                    .pipe(pd.merge, (stats_features.query('AwayTeam==team')
                                                        .drop(columns=['team'])
                                                        .rename(columns={col: 'f_' + col + 'Away' for col in stats_features.columns 
                                                                         if col not in non_features})), on=['game_index'])
                                    .pipe(pd.merge, df[['game_index', 'winner']], on='game_index')
                                    .dropna())
    return stats_features_restructured

#restructure_stats_features(stats_features).head()

In [422]:
def optimise_alpha(features):
    le = LabelEncoder()
    y = le.fit_transform(features.winner) # Encode the result from away, draw, home win to 0, 1, 2
    X = features[[col for col in features.columns if col.startswith('f_')]] # Only get the features - these all start with f_
    lr = LogisticRegression()
    
    kfold = StratifiedKFold(n_splits=5)
    ave_cv_score = cross_val_score(lr, X, y, scoring='neg_log_loss', cv=kfold).mean()
    return ave_cv_score

In [423]:
'''
best_score = np.float('inf')
best_span = 0
cv_scores = []

# Iterate over a range of spans
for span in range(1,16):
    stats_features = create_stats_features_ema(stats, span=span)
    restructured_stats_features = restructure_stats_features(stats_features)
    cv_score = optimise_alpha(restructured_stats_features)
    cv_scores.append(cv_score)
    
    if cv_score * -1 < best_score:
        best_score = cv_score * -1
        best_span = span '''

"\nbest_score = np.float('inf')\nbest_span = 0\ncv_scores = []\n\n# Iterate over a range of spans\nfor span in range(1,16):\n    stats_features = create_stats_features_ema(stats, span=span)\n    restructured_stats_features = restructure_stats_features(stats_features)\n    cv_score = optimise_alpha(restructured_stats_features)\n    cv_scores.append(cv_score)\n    \n    if cv_score * -1 < best_score:\n        best_score = cv_score * -1\n        best_span = span "

In [424]:
'''
plt.style.use('ggplot')
plt.plot(list(range(1, 16)), (pd.Series(cv_scores)*-1)) # Plot our results

plt.title("Optimising alpha")
plt.xlabel("Span")
plt.ylabel("Log Loss")
plt.show()

print("Our lowest log loss ({:2f}) occurred at a span of {}".format(best_score, best_span))'''

'\nplt.style.use(\'ggplot\')\nplt.plot(list(range(1, 16)), (pd.Series(cv_scores)*-1)) # Plot our results\n\nplt.title("Optimising alpha")\nplt.xlabel("Span")\nplt.ylabel("Log Loss")\nplt.show()\n\nprint("Our lowest log loss ({:2f}) occurred at a span of {}".format(best_score, best_span))'

In [453]:
stats = df.copy()
stats_features = create_stats_features_ema(stats, span=14)
feature_df = stats_features

In [454]:
feature_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1002 entries, 2018-12-15-13-south-florida to 2018-11-06-20-wisconsin
Data columns (total 45 columns):
team                                      1002 non-null object
winner                                    1002 non-null object
HomeTeam                                  1002 non-null object
AwayTeam                                  1002 non-null object
date                                      1002 non-null object
game_index                                1002 non-null float64
home_points                               1002 non-null float64
away_points                               1002 non-null float64
away_assist_percentage                    944 non-null float64
away_block_percentage                     944 non-null float64
away_defensive_rating                     944 non-null float64
away_defensive_rebound_percentage         944 non-null float64
away_effective_field_goal_percentage      944 non-null float64
away_field_goal_percentage     

In [455]:
# Create differential df - this df is the home features - the away features
away_cols=['away_assist_percentage', 'away_block_percentage','away_defensive_rating','away_defensive_rebound_percentage',
           'away_effective_field_goal_percentage','away_field_goal_percentage','away_free_throw_attempt_rate',
           'away_free_throw_percentage' ,'away_offensive_rating','away_offensive_rebound_percentage','away_steal_percentage',
           'away_three_point_attempt_rate', 'away_three_point_field_goal_percentage','away_total_rebound_percentage',
           'away_true_shooting_percentage','away_turnover_percentage','away_two_point_field_goal_percentage','away_win_percentage'] 

home_cols=[]
for col in away_cols:
    s = 'home_'+col[-(len(col)-5):]
    home_cols.append(s)
    
non_diff_cols =['team','winner','HomeTeam', 'AwayTeam', 'date', 'game_index','home_points','away_points','pace']

diff_df = feature_df[non_diff_cols].copy()

for col in home_cols:
    diff_df[col+'_diff'] = feature_df[col] - feature_df['away_'+col[-(len(col)-5):]]

In [456]:
diff_df.shape

(1002, 27)

In [457]:
cols = ['home_assist_percentage_diff',
 'home_block_percentage_diff',
 'home_defensive_rating_diff',
 'home_defensive_rebound_percentage_diff',
 'home_effective_field_goal_percentage_diff',
 'home_field_goal_percentage_diff',
 'home_free_throw_attempt_rate_diff',
 'home_free_throw_percentage_diff',
 'home_offensive_rating_diff',
 'home_offensive_rebound_percentage_diff',
 'home_steal_percentage_diff',
 'home_three_point_attempt_rate_diff',
 'home_three_point_field_goal_percentage_diff',
 'home_total_rebound_percentage_diff',
 'home_true_shooting_percentage_diff',
 'home_turnover_percentage_diff',
 'home_two_point_field_goal_percentage_diff',
 'home_win_percentage_diff','pace']

In [458]:
##Data Scaling

In [459]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

In [460]:
feats =  pd.DataFrame(scale.fit_transform(diff_df[cols]),
                           columns=cols,index=diff_df.index)

In [461]:
feats

,home_assist_percentage_diff,home_block_percentage_diff,home_defensive_rating_diff,home_defensive_rebound_percentage_diff,home_effective_field_goal_percentage_diff,home_field_goal_percentage_diff,home_free_throw_attempt_rate_diff,home_free_throw_percentage_diff,home_offensive_rating_diff,home_offensive_rebound_percentage_diff,home_steal_percentage_diff,home_three_point_attempt_rate_diff,home_three_point_field_goal_percentage_diff,home_total_rebound_percentage_diff,home_true_shooting_percentage_diff,home_turnover_percentage_diff,home_two_point_field_goal_percentage_diff,home_win_percentage_diff,pace
2018-12-15-13-south-florida,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-18-18-georgetown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-20-19-appalachian-state,-0.342143,-0.648222,-0.283362,1.238607,1.023456,0.868634,0.843831,-3.771042,0.283362,1.238697,-0.002495,-2.225279,2.997293,1.096664,0.348607,1.914138,-1.134997,2.688054,2.774446
2018-12-30-15-saint-louis,0.761891,-0.025501,-2.401840,1.722736,2.063062,1.781746,0.897264,-0.620378,2.401840,1.722858,1.189536,-1.206666,3.156925,2.040034,1.847538,0.522694,-0.013112,2.851664,2.767292
2018-12-04-19-appalachian-state,0.297502,0.931206,-3.043243,2.254761,2.360976,2.250729,0.390221,-1.019760,3.043243,2.254918,0.977130,-1.170850,2.592345,2.448975,1.971792,-0.634414,0.788217,2.626211,1.895616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-11-22-13-oklahoma,-0.925112,-0.556347,0.092139,0.062817,-0.011622,0.256691,0.619871,0.420091,-0.092139,0.065529,-0.027328,0.105053,-1.394405,0.406615,0.131212,0.534777,0.940446,0.407055,-0.505869
2018-11-23-14-wisconsin,-1.594440,-0.075713,0.844337,-0.029547,-0.365242,0.093300,0.483560,-0.259986,-0.844337,-0.027205,-0.380748,-0.272782,-1.954895,0.157649,-0.337592,1.232450,1.059679,0.013822,-0.640370
2018-11-27-21-wisconsin,-1.243237,0.205396,1.130780,-0.173764,-0.261988,0.223182,0.001375,-0.441890,-1.130780,-0.171746,-0.956451,-0.498791,-1.903451,0.167680,-0.351142,1.778351,1.051286,-0.285081,-1.010351
2018-11-30-20-iowa,-0.562382,0.199463,1.042303,-0.413676,-0.126761,0.158788,0.333972,-0.742396,-1.042303,-0.411946,-1.051858,-0.128435,-1.351592,-0.155431,-0.237324,1.568483,0.738401,-0.338383,-0.785560


In [462]:
diff_df[cols]=feats[cols]

In [467]:
le = LabelEncoder()
diff_df.winner = le.fit_transform(diff_df.winner)

In [468]:
diff_df

,team,winner,HomeTeam,AwayTeam,date,game_index,home_points,away_points,pace,home_assist_percentage_diff,home_block_percentage_diff,home_defensive_rating_diff,home_defensive_rebound_percentage_diff,home_effective_field_goal_percentage_diff,home_field_goal_percentage_diff,home_free_throw_attempt_rate_diff,home_free_throw_percentage_diff,home_offensive_rating_diff,home_offensive_rebound_percentage_diff,home_steal_percentage_diff,home_three_point_attempt_rate_diff,home_three_point_field_goal_percentage_diff,home_total_rebound_percentage_diff,home_true_shooting_percentage_diff,home_turnover_percentage_diff,home_two_point_field_goal_percentage_diff,home_win_percentage_diff
2018-12-15-13-south-florida,APPALACHIAN-STATE,1,SOUTH-FLORIDA,APPALACHIAN-STATE,"December 15, 2018",0.0,76.0,69.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-18-18-georgetown,APPALACHIAN-STATE,1,GEORGETOWN,APPALACHIAN-STATE,"December 18, 2018",1.0,83.0,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-20-19-appalachian-state,APPALACHIAN-STATE,1,APPALACHIAN-STATE,Milligan\n\t\t\t,"December 20, 2018",2.0,99.0,60.0,2.774446,-0.342143,-0.648222,-0.283362,1.238607,1.023456,0.868634,0.843831,-3.771042,0.283362,1.238697,-0.002495,-2.225279,2.997293,1.096664,0.348607,1.914138,-1.134997,2.688054
2018-12-30-15-saint-louis,APPALACHIAN-STATE,1,SAINT-LOUIS,APPALACHIAN-STATE,"December 30, 2018",3.0,83.0,55.0,2.767292,0.761891,-0.025501,-2.401840,1.722736,2.063062,1.781746,0.897264,-0.620378,2.401840,1.722858,1.189536,-1.206666,3.156925,2.040034,1.847538,0.522694,-0.013112,2.851664
2018-12-04-19-appalachian-state,APPALACHIAN-STATE,1,APPALACHIAN-STATE,HOWARD,"December 4, 2018",4.0,100.0,86.0,1.895616,0.297502,0.931206,-3.043243,2.254761,2.360976,2.250729,0.390221,-1.019760,3.043243,2.254918,0.977130,-1.170850,2.592345,2.448975,1.971792,-0.634414,0.788217,2.626211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-11-22-13-oklahoma,WISCONSIN,0,OKLAHOMA,WISCONSIN,"November 22, 2018",29.0,58.0,78.0,-0.505869,-0.925112,-0.556347,0.092139,0.062817,-0.011622,0.256691,0.619871,0.420091,-0.092139,0.065529,-0.027328,0.105053,-1.394405,0.406615,0.131212,0.534777,0.940446,0.407055
2018-11-23-14-wisconsin,WISCONSIN,0,WISCONSIN,VIRGINIA,"November 23, 2018",30.0,46.0,53.0,-0.640370,-1.594440,-0.075713,0.844337,-0.029547,-0.365242,0.093300,0.483560,-0.259986,-0.844337,-0.027205,-0.380748,-0.272782,-1.954895,0.157649,-0.337592,1.232450,1.059679,0.013822
2018-11-27-21-wisconsin,WISCONSIN,1,WISCONSIN,NORTH-CAROLINA-STATE,"November 27, 2018",31.0,79.0,75.0,-1.010351,-1.243237,0.205396,1.130780,-0.173764,-0.261988,0.223182,0.001375,-0.441890,-1.130780,-0.171746,-0.956451,-0.498791,-1.903451,0.167680,-0.351142,1.778351,1.051286,-0.285081
2018-11-30-20-iowa,WISCONSIN,0,IOWA,WISCONSIN,"November 30, 2018",32.0,66.0,72.0,-0.785560,-0.562382,0.199463,1.042303,-0.413676,-0.126761,0.158788,0.333972,-0.742396,-1.042303,-0.411946,-1.051858,-0.128435,-1.351592,-0.155431,-0.237324,1.568483,0.738401,-0.338383


In [469]:
corr = diff_df.corr()

In [470]:
corr.to_csv(r"C:\Users\socst\Documents\Python Scripts\Betting Models\test.csv")

In [442]:
diff_df = input_df.dropna().drop_duplicates()

In [443]:
FIELDS_TO_DROP = ['away_points', 'home_points', 'winner','HomeTeam', 'AwayTeam','date','game_index','team']

In [444]:
X = input_df.drop(FIELDS_TO_DROP, 1).dropna().drop_duplicates()
y = input_df[['home_points', 'away_points']].values

In [445]:
print(X.shape)
print(y.shape)

(944, 75)
(944, 2)


In [446]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
parameters = {'bootstrap': False,
              'min_samples_leaf': 3,
              'n_estimators': 50,
              'min_samples_split': 10,
              'max_features': 'sqrt',
              'max_depth': 6}
model = RandomForestRegressor(**parameters)
model.fit(X_train, y_train)
print(model.predict(X_test).astype(int), y_test)

[[70 65]
 [70 66]
 [73 67]
 [69 64]
 [77 73]
 [76 68]
 [69 65]
 [77 68]
 [71 65]
 [82 75]
 [80 76]
 [71 65]
 [69 65]
 [68 62]
 [75 67]
 [77 63]
 [72 67]
 [71 65]
 [69 63]
 [74 64]
 [77 71]
 [68 64]
 [81 71]
 [75 67]
 [76 74]
 [68 63]
 [74 68]
 [75 66]
 [79 69]
 [72 66]
 [67 62]
 [68 61]
 [69 63]
 [77 74]
 [82 76]
 [70 64]
 [78 65]
 [72 66]
 [68 64]
 [72 66]
 [72 67]
 [69 65]
 [75 68]
 [73 66]
 [70 64]
 [70 64]
 [68 63]
 [75 68]
 [71 65]
 [78 72]
 [81 73]
 [82 67]
 [71 65]
 [70 66]
 [83 72]
 [72 66]
 [76 65]
 [70 64]
 [73 65]
 [69 64]
 [72 66]
 [77 67]
 [73 65]
 [70 64]
 [75 69]
 [76 66]
 [69 65]
 [69 64]
 [74 80]
 [82 74]
 [74 65]
 [75 72]
 [67 62]
 [69 65]
 [71 66]
 [78 81]
 [74 66]
 [75 66]
 [72 65]
 [79 70]
 [78 67]
 [77 66]
 [68 64]
 [68 63]
 [70 67]
 [68 65]
 [70 65]
 [69 64]
 [73 67]
 [69 63]
 [71 66]
 [71 64]
 [73 69]
 [71 66]
 [71 66]
 [69 64]
 [69 65]
 [70 66]
 [77 76]
 [69 63]
 [68 64]
 [79 71]
 [69 64]
 [70 66]
 [81 68]
 [69 65]
 [72 66]
 [73 65]
 [72 67]
 [73 66]
 [74 65]
 

In [447]:
preds = model.predict(X_test).astype(int)
test = pd.DataFrame(y_test,columns =['testH','testA'])
test2 = pd.DataFrame(preds,columns =['predH','predA'])
test['predH'] = test2['predH']
test['predA'] = test2['predA']

In [448]:
test

,testH,testA,predH,predA
0,91.0,74.0,70,65
1,93.0,91.0,70,66
2,99.0,94.0,73,67
3,74.0,73.0,69,64
4,99.0,60.0,77,73
...,...,...,...,...
231,79.0,61.0,74,68
232,91.0,76.0,71,66
233,43.0,61.0,75,68
234,88.0,79.0,78,72
